# Biweekly Report

# Jake Watts

# Random Erasing + MLP-Mixer

Here I combine code for random erasing data processing with the MLP-Mixer architecture. I was interested in using random erasing with the MLP_Mixer because the random erasing paper compared performance when adding the model to CNN models so I wanted to see how adding it in to a different type of model would effect the performance. I first train a model with no random erasing, then one with random erasing and finally a thrid model with pixel-level random erasing to see how each performs.

The code used for the MLP_Mixer comes from https://github.com/sayakpaul/MLP-Mixer-CIFAR10/blob/main/ResNet20.ipynb. I reworked the code so that it could work with the random eraser function.

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import io
import os

In [ ]:
try: 
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: 
    strategy = tf.distribute.MirroredStrategy() 

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1


Defining model hyper-parameters.

In [ ]:
RESIZE_TO = 72
PATCH_SIZE = 9

NUM_MIXER_LAYERS = 4
HIDDEN_SIZE = 128
MLP_SEQ_DIM = 64
MLP_CHANNEL_DIM = 128

EPOCHS = 25
BATCH_SIZE = 512 * strategy.num_replicas_in_sync

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


Function for data-augmentation and imaging resizing.

In [25]:
def get_augmentation_layers():
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.Normalization(),
            layers.experimental.preprocessing.Resizing(RESIZE_TO, RESIZE_TO),
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(factor=0.02),
            layers.experimental.preprocessing.RandomZoom(
                height_factor=0.2, width_factor=0.2
            ),
        ],
        name="data_augmentation",
    )
    # Compute the mean and the variance of the training data for normalization.
    data_augmentation.layers[0].adapt(x_train)
    
    return data_augmentation

Below is the code for the MLP-Mixer model which was based off the code from Appendix E of the MLP-Mixer paper (https://arxiv.org/pdf/2105.01601.pdf)

In [ ]:
def mlp_block(x, mlp_dim):
    x = layers.Dense(mlp_dim)(x)
    x = tf.nn.gelu(x)
    return layers.Dense(x.shape[-1])(x)

def mixer_block(x, tokens_mlp_dim, channels_mlp_dim):
    y = layers.LayerNormalization()(x)
    y = layers.Permute((2, 1))(y)
    
    token_mixing = mlp_block(y, tokens_mlp_dim)
    token_mixing = layers.Permute((2, 1))(token_mixing)
    x = layers.Add()([x, token_mixing])
    
    y = layers.LayerNormalization()(x)
    channel_mixing = mlp_block(y, channels_mlp_dim)
    output = layers.Add()([x, channel_mixing])
    return output

def mlp_mixer(x, num_blocks, patch_size, hidden_dim, 
              tokens_mlp_dim, channels_mlp_dim,
              num_classes=10):
    x = layers.Conv2D(hidden_dim, kernel_size=patch_size,
                      strides=patch_size, padding="valid")(x)
    x = layers.Reshape((x.shape[1]*x.shape[2], x.shape[3]))(x)

    for _ in range(num_blocks):
        x = mixer_block(x, tokens_mlp_dim, channels_mlp_dim)
    
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.GlobalAveragePooling1D()(x)
    return layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

Creating Model

In [28]:
def create_mlp_mixer():
    data_augmentation = get_augmentation_layers()
    
    inputs = layers.Input(shape=(32, 32, 3))
    augmented = data_augmentation(inputs)
    outputs = mlp_mixer(augmented, NUM_MIXER_LAYERS,
                        PATCH_SIZE, HIDDEN_SIZE, 
                        MLP_SEQ_DIM, MLP_CHANNEL_DIM)
    return tf.keras.Model(inputs, outputs, name="mlp_mixer")

In [32]:
# We need to instantiate the model here for the callback below.
with strategy.scope():
    mlp_mixer_classifier = create_mlp_mixer()

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7ff4f1bf8830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Setting up model saving and progress

In [33]:
file_writer_cm = tf.summary.create_file_writer(os.path.join(os.getcwd(), 'saved_models'))

def plot_to_image(figure):
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def log_progression(epoch, logs):
    projections = mlp_mixer_classifier.layers[2].get_weights()[0]
    p_min, p_max = projections.min(), projections.max()
    projections = (projections - p_min) / (p_max - p_min)

    figure = plt.figure(figsize=(10, 10))
    n_filters, ix = 128, 1
    for i in range(n_filters):
        projection = projections[:, :, :, i]
        plt.subplot(8, 16, ix)
        plt.axis("off")
        plt.imshow(projection)
        ix += 1

    progress_image = plot_to_image(figure)
    with file_writer_cm.as_default():
        tf.summary.image("Progression", progress_image, step=epoch)

In [ ]:
Code to compile and train model

In [ ]:
def run_experiment(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001 * strategy.num_replicas_in_sync)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )
    
    progress_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_progression)

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.1,
        callbacks=[checkpoint_callback, progress_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, top_1_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(top_1_accuracy * 100, 2)}%")
    
    return history, model

After training the MLP-Mixer model for 25 epocjs the test accuracy is 69.47%. The accuracy is much lower than the ResNet model however this is not too much of a surprise as that model was much deeper and trained for longer. The MLP-Mixer also tends to have the highest performance for very large datasets. However I am more interested in seeing if the model performance changes with the addition of random erasing.

In [ ]:
with strategy.scope():
    history, model = run_experiment(mlp_mixer_classifier)
    model.save(f"mlp_mixer_{NUM_MIXER_LAYERS}")

Epoch 1/25
88/88 [==============================] - 26s 223ms/step - loss: 1.9026 - accuracy: 0.3064 - val_loss: 1.6054 - val_accuracy: 0.4138
Epoch 2/25
88/88 [==============================] - 18s 199ms/step - loss: 1.5305 - accuracy: 0.4481 - val_loss: 1.4184 - val_accuracy: 0.4818
Epoch 3/25
88/88 [==============================] - 17s 198ms/step - loss: 1.3988 - accuracy: 0.4953 - val_loss: 1.3117 - val_accuracy: 0.5254
Epoch 4/25
88/88 [==============================] - 17s 198ms/step - loss: 1.3129 - accuracy: 0.5280 - val_loss: 1.2421 - val_accuracy: 0.5450
Epoch 5/25
88/88 [==============================] - 17s 198ms/step - loss: 1.2541 - accuracy: 0.5506 - val_loss: 1.2062 - val_accuracy: 0.5592
Epoch 6/25
88/88 [==============================] - 17s 198ms/step - loss: 1.2029 - accuracy: 0.5734 - val_loss: 1.1619 - val_accuracy: 0.5820
Epoch 7/25
88/88 [==============================] - 17s 198ms/step - loss: 1.1562 - accuracy: 0.5892 - val_loss: 1.1413 - val_accuracy: 0.5900

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Here I add the function for the random eraser and apply it to the data and retrain the data to see how it affects model performance. The rest of the model is unchanged.

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
eraser = get_random_eraser(pixel_level = False)

for i in range(len(x_train)):
  eraser(x_train[i])

In [ ]:
# We need to instantiate the model here for the callback below.
with strategy.scope():
    mlp_mixer_classifier = create_mlp_mixer()

In [ ]:
def run_experiment(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001 * strategy.num_replicas_in_sync)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )
    
    progress_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_progression)

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.1,
        callbacks=[checkpoint_callback, progress_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, top_1_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(top_1_accuracy * 100, 2)}%")
    
    return history, model

Surprisingly, the model's testing accuracy decreases from 69.47% to 67.92% after adding random erasing as I expected it to increase. It is possible I did not train for enough epochs for the model to adapt to the random erasing.

---



In [ ]:
with strategy.scope():
    history, model = run_experiment(mlp_mixer_classifier)
    model.save(f"mlp_mixer_{NUM_MIXER_LAYERS}")

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/25
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tens

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Finally I train the model for a third and final time using pixel_level random erasing which yield higher accuracy than regular random erasing with the ResNet model.

In [26]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
pixel_level = True
eraser = get_random_eraser(pixel_level = True)

for i in range(len(x_train)):
  eraser(x_train[i])

In [34]:
# We need to instantiate the model here for the callback below.
with strategy.scope():
    mlp_mixer_classifier = create_mlp_mixer()

In [35]:
def run_experiment(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001 * strategy.num_replicas_in_sync)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True ,
        save_weights_only=True,
    )
    
    progress_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_progression)

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.1,
        callbacks=[checkpoint_callback, progress_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, top_1_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(top_1_accuracy * 100, 2)}%")
    
    return history, model

With a testing accuracy of 68.4% this model performs better than regular random erasing which was expected but worse than no random erasing.

In [36]:
with strategy.scope():
    history, model = run_experiment(mlp_mixer_classifier)
    model.save(f"mlp_mixer_{NUM_MIXER_LAYERS}")

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/25
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
88/88 [==============================] - 33s 113ms/step - loss: 1.9258 - accuracy: 0.2953 - val_loss: 1.6788 - val_accuracy: 0.3886
Epoch 2/25
88/88 

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# Summary

The addition of random erasing data augmentation to the MLP-Mixer did not increase the perfromance as I expected. This could be for several reasons. One could be I did not train the model for long enough. Another could be that the implementation of random erasing could be improved by adjusting the hyper-parameters or making changes to the model architecture. Or finally it could be that random erasing is more beneficial for convultional neural networks.

Despite the results being different from anticipated, implementing this model gave me a much better understanding of MLP-Mixers and also adding in code from another source was definitely very challenging but helped me better understand the Tensorflow frameworks and python in general.